In [ ]:
import pandas as pd
import numpy as np
import altair as alt

In [ ]:
# import raw data
raw = pd.read_csv('ESGData.csv')

# merge the raw data with the Topics of Indicator Names
code = pd.read_csv('ESGSeries.csv').loc[:, ['Topic', 'Indicator Name']]
code['Topic'] = code['Topic'].str.split(':', expand = True)[0]
raw_merge = pd.merge(raw, code, how = 'left', on = 'Indicator Name')

# Create a region dataframe
region = pd.DataFrame({
    'Country Name': ['China', 'Japan', 'India','United Kingdom','Germany','Russian Federation','France', 'Italy', 
           'Egypt, Arab Rep.','South Africa', 'Congo, Dem. Rep.','United States', 'Brazil', 'Argentina',
           'Canada', 'Australia', 'New Zealand','Fiji'],
    'Region': ['Asia', 'Asia', 'Asia','Europe','Europe','Europe','Europe', 'Europe', 
               'Africa', 'Africa', 'Africa','North America', 'South America', 'South America',
           'North America', 'Oceania', 'Oceania','Oceania']
})

# only selected the needed rows and coulmns
country = ['China', 'Japan', 'India','United Kingdom','Germany','Russian Federation','France', 'Italy', 
           'Egypt, Arab Rep.', 'South Africa', 'Congo, Dem. Rep.','United States', 'Brazil', 'Argentina',
           'Canada', 'Australia', 'New Zealand', 'Fiji']
raw1 = raw_merge.loc[:, ['Country Name', 'Country Code', 'Indicator Name', 'Topic',
                        '2001', '2002', '2003', '2004', '2005', 
                        '2006', '2007', '2008', '2009', '2010', 
                        '2011', '2012', '2013', '2014', '2015', 
                        '2016', '2017', '2018', '2019', '2020']]
raw1 = raw1.loc[raw1['Country Name'].isin(country)]
raw1['Topic'] = raw1['Topic'].fillna('Social')
raw1 = raw1.sort_values('Topic')
raw2 = pd.merge(raw1, region, how = 'left', on = 'Country Name')

# melt and pivot the columns
clean = raw2.drop(
    columns = 'Country Code'
).melt(
    id_vars = ['Region', 'Country Name', 'Topic', 'Indicator Name'],
    var_name = 'Year',
    value_name = 'level'
).pivot(
    index = ['Region', 'Country Name', 'Year'],
    columns = ['Topic', 'Indicator Name'],
    values = 'level'
)

# we only want to choose the columns with no missing values
# tidied dataset
clean2 = clean.loc[:, (clean.isna().mean() <= 0.0)]

In [ ]:
# rename the columns names
col_names = {
        'Forest area (% of land area)': 'fore_area',
        'Adjusted savings: net forest depletion (% of GNI)': 'fore_dep',
        'Adjusted savings: natural resources depletion (% of GNI)': 'natu_res_dep',
        'Population density (people per sq. km of land area)': 'pop_denst',
        'Ratio of female to male labor force participation rate (%) (modeled ILO estimate)' : 'rate_labor',
        'GDP growth (annual %)': 'gdp_grow',
        'Unemployment, total (% of total labor force) (modeled ILO estimate)': 'unemp_rate',
        'Life expectancy at birth, total (years)': 'life_exp',
        'Access to electricity (% of population)': 'acce_electr',
        'Mortality rate, under-5 (per 1,000 live births)': 'mortal_rate',
        'Access to clean fuels and technologies for cooking (% of population)': 'acce_fuel_tech',
        'Population ages 65 and above (% of total population)': 'pop_65',
        'Fertility rate, total (births per woman)': 'ferti_rate'
    }

# selected the needed variables
data = clean2.rename(
    columns = col_names
).drop(
    columns = [('Social', 'mortal_rate'), 
          ('Social', 'pop_65'),
          ('Social', 'ferti_rate')]
)
data

Topic                               Environment            Governance  \
Indicator Name                        pop_denst  fore_area   gdp_grow   
Region        Country Name     Year                                     
Africa        Congo, Dem. Rep. 2001   21.361917  63.177257  -2.100173   
                               2002   21.998487  62.880395   2.947765   
                               2003   22.683921  62.583534   5.577822   
                               2004   23.408777  62.286672   6.738374   
                               2005   24.166160  61.989811   6.135151   
...                                         ...        ...        ...   
South America Brazil           2016   24.666141  60.071033  -3.275917   
                               2017   24.866038  59.832881   1.322869   
                               2018   25.061715  59.708428   1.783667   
                               2019   25.250776  59.558526   1.411153   
                               2020   25.431425  59.417478  -4.059048   

Topic                                               Social           \
Indicator Name                      rate_labor acce_electr life_exp   
Region        Country Name     Year                                   
Africa        Congo, Dem. Rep. 2001  96.724567    7.314364   50.667   
                               2002  96.617267    7.915845   51.385   
                               2003  96.539211    8.512090   52.144   
                               2004  96.488953    9.105449   52.917   
                               2005  96.465138    6.000000   53.675   
...                                        ...         ...      ...   
South America Brazil           2016  70.854286   99.699997   75.230   
                               2017  72.456906   99.800003   75.456   
                               2018  73.004277   99.699997   75.672   
                               2019  73.910763   99.800003   75.881   
                               2020  70.792654  100.000000   76.084   

Topic                                                          
Indicator Name                      unemp_rate acce_fuel_tech  
Region        Country Name     Year                            
Africa        Congo, Dem. Rep. 2001      2.888            1.2  
                               2002      2.871            1.4  
                               2003      2.860            1.6  
                               2004      2.853            1.9  
                               2005      2.850            2.1  
...                                        ...            ...  
South America Brazil           2016     11.600           95.7  
                               2017     12.820           95.7  
                               2018     12.330           95.8  
                               2019     11.930           95.8  
                               2020     13.690           95.9  

[360 rows x 8 columns]

In [ ]:
data.reset_index()

Topic                  Region      Country Name  Year Environment             \
Indicator Name                                          pop_denst  fore_area   
0                      Africa  Congo, Dem. Rep.  2001   21.361917  63.177257   
1                      Africa  Congo, Dem. Rep.  2002   21.998487  62.880395   
2                      Africa  Congo, Dem. Rep.  2003   22.683921  62.583534   
3                      Africa  Congo, Dem. Rep.  2004   23.408777  62.286672   
4                      Africa  Congo, Dem. Rep.  2005   24.166160  61.989811   
..                        ...               ...   ...         ...        ...   
355             South America            Brazil  2016   24.666141  60.071033   
356             South America            Brazil  2017   24.866038  59.832881   
357             South America            Brazil  2018   25.061715  59.708428   
358             South America            Brazil  2019   25.250776  59.558526   
359             South America            Brazil  2020   25.431425  59.417478   

Topic          Governance                 Social                      \
Indicator Name   gdp_grow rate_labor acce_electr life_exp unemp_rate   
0               -2.100173  96.724567    7.314364   50.667      2.888   
1                2.947765  96.617267    7.915845   51.385      2.871   
2                5.577822  96.539211    8.512090   52.144      2.860   
3                6.738374  96.488953    9.105449   52.917      2.853   
4                6.135151  96.465138    6.000000   53.675      2.850   
..                    ...        ...         ...      ...        ...   
355             -3.275917  70.854286   99.699997   75.230     11.600   
356              1.322869  72.456906   99.800003   75.456     12.820   
357              1.783667  73.004277   99.699997   75.672     12.330   
358              1.411153  73.910763   99.800003   75.881     11.930   
359             -4.059048  70.792654  100.000000   76.084     13.690   

Topic                          
Indicator Name acce_fuel_tech  
0                         1.2  
1                         1.4  
2                         1.6  
3                         1.9  
4                         2.1  
..                        ...  
355                      95.7  
356                      95.7  
357                      95.8  
358                      95.8  
359                      95.9  

[360 rows x 11 columns]

In [ ]:
# correlation
corr_mx = diatoms.drop(columns = ['Depth', 'Age']).corr() 
corr_mx

NameError: name 'diatoms' is not defined

In [ ]:
# Heat Map
corr_mx = x_mx.corr()
alt.Chart().mark_rect().encode(
    x = alt.X('col', title = '', sort = {'field': 'Correlation', 'order': 'ascending'}), 
    y = alt.Y('row', title = '', sort = {'field': 'Correlation', 'order': 'ascending'}),
    color = alt.Color('Correlation', 
                      scale = alt.Scale(scheme = 'blueorange', # diverging gradient
                                        domain = (-1, 1), # ensure white = 0
                                        type = 'sqrt'), # adjust gradient scale
                     legend = alt.Legend(tickCount = 5)) # add ticks to colorbar at 0.5 for reference
).properties(width = 300, height = 300)

NameError: name 'x_mx' is not defined

In [ ]:
# Average value for every 5 years, and compute each values into 3 difference levels (1-low, 2-medium, 3-high)


In [ ]:
# multiple linear regression


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=59144ecd-8932-4795-927c-6a7d21b33446' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>